## Scraping (bloqué)

In [1]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import time
import json

In [2]:
cookies = {"earchives_vaucluse_fr_bach_cookie":"%7B%22map%22%3Afalse%2C%22daterange%22%3Afalse%2C%22piwik_ignore%22%3A%221%22%2C%22social_ignore%22%3A%221%22%7D", "PHPSESSID":"d86cef0537293c70af58667a920c14ef"}

In [3]:
headers= {"User-Agent": "Mozilla/5.0"}
response = requests.get("https://earchives.vaucluse.fr/archives/show/FRAD084_IR0001612_de-11", headers=headers, cookies=cookies)
print(response.text)

You got banned permanently from this server.


## Scraping Selenium

In [8]:
# !source ./.venv/vin/activate

In [5]:
# !pip install jupyter

In [4]:
# !pip install selenium

In [ ]:
import json
with open("/Users/mathieu/Documents/CODE/hackathon/vaucluse/data/vaucluse_urls_notices.json", 'r', encoding="utf-8") as f:
    vaucluse_urls_notices = json.load(f)

In [ ]:
with open("/Users/mathieu/Documents/CODE/hackathon/vaucluse/data/vaucluse_urls_photos.json", 'r', encoding="utf-8") as f:
    vaucluse_urls_photos = json.load(f)

In [15]:
vaucluse_urls_notices2 = vaucluse_urls_notices[:10]
vaucluse_urls_photos2 = vaucluse_urls_photos[:10]
print(vaucluse_urls_notices2)
print(vaucluse_urls_photos2)

['https://earchives.vaucluse.fr/archives/show/FRAD084_IR0001612_de-11', 'https://earchives.vaucluse.fr/archives/show/FRAD084_IR0001612_de-12', 'https://earchives.vaucluse.fr/archives/show/FRAD084_IR0001612_de-13', 'https://earchives.vaucluse.fr/archives/show/FRAD084_IR0001612_de-14', 'https://earchives.vaucluse.fr/archives/show/FRAD084_IR0001612_de-15', 'https://earchives.vaucluse.fr/archives/show/FRAD084_IR0001612_de-22', 'https://earchives.vaucluse.fr/archives/show/FRAD084_IR0001612_de-24', 'https://earchives.vaucluse.fr/archives/show/FRAD084_IR0001612_de-23', 'https://earchives.vaucluse.fr/archives/show/FRAD084_IR0001612_de-25', 'https://earchives.vaucluse.fr/archives/show/FRAD084_IR0001612_de-20']
['https://earchives.vaucluse.fr/archives/show/FRAD084_IR0001612_de-11#relative_documents', 'https://earchives.vaucluse.fr/archives/show/FRAD084_IR0001612_de-12#relative_documents', 'https://earchives.vaucluse.fr/archives/show/FRAD084_IR0001612_de-13#relative_documents', 'https://earchives

In [26]:
data = {
    "File name": [],
    "Raw text": [],
    "City": [],
    "Hamlet": [],
    "Monument": [],
    "Other": [],
    "Dates": [],
    "Description physique": [],
    "Dimensions": [],
    "Données techniques": [],
    "Informations sur les modalités d'entrée": [],
    "Matière": [],
    "Nom géographique": [],
    "Édifice": [],
    "Lien_notice": [],
    "Lien_photo": []
}

##### récupérer tout le html

In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import csv

service = Service("/opt/homebrew/bin/geckodriver")  # Apple Silicon
# service = Service("/usr/local/bin/geckodriver")   # Intel

driver = webdriver.Firefox(service=service)
wait = WebDriverWait(driver, 10)

photos_content_html = []

for url_notice, url_photos in zip(vaucluse_urls_notices2, vaucluse_urls_photos2):
    # --- photo page: grab image link from <a href=...> ---
    driver.get(url_photos)
    content_el = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.content")))
    photos_content_html.append(content_el.get_attribute("innerHTML"))

    # --- notice page: grab div.content HTML ---
    driver.get(url_notice)

driver.quit()

In [21]:
photos_content_html

['\n        <ul class="ui-tabs-nav ui-helper-reset ui-helper-clearfix ui-widget-header ui-corner-all" role="tablist">\n<li class="ui-state-default ui-corner-top" role="tab" tabindex="-1" aria-controls="de-11" aria-labelledby="ui-id-1" aria-selected="false"><a href="#de-11" class="ui-tabs-anchor" role="presentation" tabindex="-1" id="ui-id-1">Description</a></li>\n<li class="ui-state-default ui-corner-top ui-tabs-active ui-state-active" role="tab" tabindex="0" aria-controls="relative_documents" aria-labelledby="ui-id-2" aria-selected="true"><a href="#relative_documents" class="ui-tabs-anchor" role="presentation" tabindex="-1" id="ui-id-2">Fichiers liés</a></li>\n</ul><div id="de-11" class="content ui-tabs-panel ui-widget-content ui-corner-bottom" aria-labelledby="ui-id-1" role="tabpanel" style="display: none;" aria-expanded="false" aria-hidden="true">\n\n\n\n<section property="dc:date" content="1920"><header><h3>Dates extrêmes</h3></header><div>[avant 1920]</div></section>\n<section cla

## récupérer métadonnées à partir des pages html

In [27]:
import re

EMPTY = " "

def extract_one(html, pattern: str) -> str:
    m = re.search(pattern, html, flags=re.IGNORECASE | re.DOTALL)
    return re.sub(r"<[^>]+>", "", m.group(1)).strip() if m else EMPTY

def extract_all(html, pattern: str):
    return re.findall(pattern, html, flags=re.IGNORECASE | re.DOTALL)

for html in photos_content_html:
    # lien photo
    photo_names = extract_all(html, r'href="https?://[^"]*/(FRAD084_[^"/]+\.jpg)[^"]*"')
    lien_photo = photo_names[-1] if photo_names else EMPTY

    # Dates extrêmes
    dates = extract_one(html, r'<section[^>]*property="dc:date"[^>]*>.*?<div>(.*?)</div>\s*</section>')

    # Description physique (tout le bloc physdesc sans le header)
    description_physique = extract_one(
        html,
        r'<section[^>]*class="physdesc"[^>]*>.*?<header>.*?</header>\s*(.*?)</section>'
    )

    # Dimensions: parfois "Dimensions", parfois "Support"
    dimensions = extract_one(html, r'<h4>\s*Dimensions\s*</h4>\s*([^<]*)')
    if dimensions == EMPTY:
        dimensions = extract_one(html, r'<h4>\s*Support\s*</h4>\s*([^<]*)')

    # Données techniques
    donnees_tech = extract_one(html, r'<h4>\s*Données techniques\s*</h4>\s*([^<]*)')

    # Informations modalités d'entrée (parfois absente)
    infos_entree = extract_one(html, r'<section[^>]*class="acqinfo"[^>]*>.*?<p>(.*?)</p>\s*</section>')

    # Matière / Édifice / Nom géographique
    matiere = extract_one(html, r'<strong>\s*Mati[eè]re\s*:</strong>\s*<a[^>]*>(.*?)</a>')
    edifice = extract_one(html, r'<strong>\s*[ÉE]difice\s*:</strong>\s*<a[^>]*>(.*?)</a>')
    nom_geo = extract_one(html, r'<strong>\s*Nom g[eé]ographique\s*:</strong>\s*<a[^>]*>(.*?)</a>')

    data["Lien_photo"].append(lien_photo)
    data["Dates"].append(dates)
    data["Description physique"].append(description_physique)
    data["Dimensions"].append(dimensions)
    data["Données techniques"].append(donnees_tech)
    data["Informations sur les modalités d'entrée"].append(infos_entree)
    data["Matière"].append(matiere)
    data["Nom géographique"].append(nom_geo)
    data["Édifice"].append(edifice)


In [34]:
data

{'File name': [],
 'Raw text': [],
 'City': [],
 'Hamlet': [],
 'Monument': [],
 'Other': [],
 'Dates': ['[avant 1920]',
  '[avant 1920]',
  '[avant 1920]',
  '[avant 1920]',
  '[avant 1920]',
  '[vers 1920]',
  '[avant 1920]',
  '[avant 1920]',
  '[avant 1920]',
  '[avant 1920]'],
 'Description physique': ['Dimensions 9 x 14 cm\n\nDonnées techniques Carte postale pleine feuille, noir et blanc. Aspect mat',
  'Dimensions 9 x 14 cm\n\nDonnées techniques Carte postale pleine feuille, noir et blanc. Aspect mat',
  'Dimensions 9 x 14 cm\n\nDonnées techniques Carte postale pleine feuille, noir et blanc. Aspect mat',
  'Dimensions 9 x 14 cm\n\nDonnées techniques Carte postale pleine feuille, noir et blanc. Aspect mat',
  'Dimensions 9 x 14 cm\n\nDonnées techniques Carte postale pleine feuille, noir et blanc. Aspect mat',
  'Dimensions 9 x 14 cm\n\nDonnées techniques Carte postale avec bordure, noir et blanc. Aspect mat',
  'Dimensions 9 x 14 cm\n\nDonnées techniques Carte postale pleine feui

In [36]:
with open("/Users/mathieu/Documents/CODE/hackathon/vaucluse/data/metadata.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2)

## Script d'Alexandre : 

In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv

service = Service("/Users/rolly/Downloads/geckodriver", )  # Change to your actual path to the geckodriver
driver = webdriver.Firefox(service=service)
wait = WebDriverWait(driver, 10)

driver.get("https://creepypasta.fandom.com/wiki/Local_Sitemap?namefrom=%22De+Profundis%22")  # Change to the actual file path or URL

all_hrefs = []
visited_links = set()  # Set to keep track of visited links

try:
    while True:
        # Find all links within the "mw-allpages-chunk" element
        chunk_links = wait.until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, '.mw-allpages-chunk a')))
        hrefs = [link.get_attribute('href') for link in chunk_links if link.get_attribute('href')]
        all_hrefs.extend(hrefs)

        # Find navigation links (both previous and next page links)
        navigation_links = wait.until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, '.mw-allpages-nav a')))
        unvisited_navigation_links = [link for link in navigation_links if link.get_attribute('href') not in visited_links]

        if unvisited_navigation_links:
            # Assuming the last unvisited link is the 'Next page' link
            next_page_link = unvisited_navigation_links[-1]  # Change here to pick the last link
            visited_links.add(next_page_link.get_attribute('href'))  # Add this link to the set of visited links
            driver.execute_script("arguments[0].click();", next_page_link)
        else:
            print("No more new pages to visit.")
            break

except Exception as e:
    print("An error occurred:", e)

finally:
    driver.quit()

# Writing to a CSV file
with open('fandom_links.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Links'])  # Writing header
    for href in all_hrefs:
        writer.writerow([href])

print("All links have been written to fandom_links.csv")